In [2]:
from data.fetch_data import fetch_data_from_db

sql = """
    SELECT stk_tkr FROM os_stk_info
"""

stk_tkr = fetch_data_from_db(query=sql)

In [5]:
from data.fetch_data import fetch_data_from_db

sql = """
    SELECT etf_tkr FROM os_index_info
"""

etf_tkr = fetch_data_from_db(query=sql)
etf_tkr_list = [item['etf_tkr'] for item in etf_tkr]

In [16]:
import pandas as pd

stk_tkr = pd.DataFrame(stk_tkr)

tkr_list = stk_tkr['stk_tkr'].to_list()

tkr_list = tkr_list + etf_tkr_list

tkr_list_str = " ".join(tkr_list)

In [17]:
import yfinance as yf

data = yf.download(tkr_list_str, period='1y')

[*********************100%***********************]  746 of 746 completed


In [18]:
data['Adj Close'].head()

,A,AA,AAON,AAPL,ABBV,ABNB,ACA,ACAD,ACB,ACCD,...,YY,Z,ZBH,ZBRA,ZG,ZIP,ZLAB,ZM,ZS,ZWS
Date,,,,,,,,,,,,,,,,,,,,,
2022-09-02,127.148201,48.754135,37.687710,154.891113,130.966827,113.639999,56.200115,16.730000,1.51,10.04,...,27.119957,34.049999,104.760529,297.600006,34.020000,16.850000,46.209999,80.790001,145.119995,26.790791
2022-09-06,128.429520,47.427628,36.438282,153.618668,132.225739,113.699997,57.027466,16.129999,1.38,9.70,...,26.194773,33.500000,104.125450,289.929993,33.509998,17.440001,48.349998,80.019997,145.029999,26.682047
2022-09-07,130.545166,47.180145,38.150455,155.040237,133.302094,116.070000,59.080898,16.850000,1.43,10.53,...,28.035789,35.220001,107.747459,295.579987,35.340000,17.680000,46.119999,81.019997,147.710007,27.334515
2022-09-08,134.279877,48.368065,37.760426,153.549103,135.041519,118.580002,58.761909,17.420000,1.50,11.21,...,26.652691,37.369999,109.940514,298.299988,37.580002,18.680000,43.330002,80.230003,154.250000,27.660748
2022-09-09,136.703445,52.090202,38.044689,156.441910,135.906418,123.480003,58.841660,17.379999,1.56,11.44,...,27.334898,37.884998,113.622070,307.839996,38.200001,19.730000,48.799999,82.620003,188.000000,28.224245


In [20]:
from tqdm.auto import tqdm

df_list = []

for tkr in tqdm(tkr_list):
    df = pd.DataFrame()
    
    df.index = data['Adj Close'].index
    df["close"] = data['Adj Close'][tkr]
    df["stk_tkr"] = tkr
    
    df_list.append(df)

  0%|          | 0/746 [00:00<?, ?it/s]

In [21]:
stk_price_df = pd.concat(df_list)

In [22]:
stk_price_df.reset_index(inplace=True)
stk_price_df.rename(columns={"Date": "date"}, inplace=True)
stk_price_df.head()

,date,close,stk_tkr
0,2022-09-02,127.148201,A
1,2022-09-06,128.429520,A
2,2022-09-07,130.545166,A
3,2022-09-08,134.279877,A
4,2022-09-09,136.703445,A


In [24]:
stk_price_df.to_csv("./data_before_db_insert/os_stk_price.csv")